In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
import torch
torch.cuda.device_count()

1

In [3]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data
from monai.data import (
    Dataset,
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


import torch

In [4]:
set_determinism(seed=1024)

In [5]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir= '/home/nada.saadi/MIS-FM/hecktor2022_cropped/MDA_PET_train_new.json'

In [6]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [7]:
train_data, valid_data = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)
len(train_data), len(valid_data)

(160, 36)

In [8]:
valid_data

[{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005',
  'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_pt.nii.gz',
  'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_gt.nii.gz',
  'fold': 0},
 {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-024',
  'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-024/MDA-024_pt.nii.gz',
  'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-024/MDA-024_gt.nii.gz',
  'fold': 0},
 {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-026',
  'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-026/MDA-026_pt.nii.gz',
  'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-026/MDA-026_gt.nii.gz',
  'fold': 0},
 {'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-053',
  'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-053/MDA-053_pt.nii.gz',
  'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-053/MDA-053_gt.nii

In [8]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [9]:
import json
import os

# Assuming 'datalist' and 'data_dir' are defined and contain the correct paths
# Example:
# datalist = '/path/to/your/datalist.json'
# data_dir = '/path/to/your/data/directory'

def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

# Replace 'json_dir' and 'data_dir' with the actual paths
train_data, valid_data = datafold_read(datalist=json_dir, basedir=data_dir, fold=0)

# Now, save the 'valid_data' to a JSON file 

json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json'  # Specify the path where you want to save the JSON
with open(json_file_path, 'w') as outfile:
    json.dump(valid_data, outfile, indent=4)

print(f"Validation data saved to {json_file_path}")


Validation data saved to /home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json


In [10]:
import json

# Path to your JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json'

# Load the JSON file
with open(json_file_path, 'r') as infile:
    data = json.load(infile)

# Count the number of entries
number_of_files = len(data)

print(f"Number of entries in the JSON file: {number_of_files}")

Number of entries in the JSON file: 36


In [8]:
num_samples = 4

train_transforms = Compose(
    [
        LoadImaged(keys=["pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["pt",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["pt",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
        #NormalizeIntensityd(keys=["ctpt"], channel_wise=True),
        RandCropByPosNegLabeld(
            keys=["pt", "seg"],
            label_key="seg",
            spatial_size=(96, 96, 96),
            pos=1,
            neg=1,
            num_samples=num_samples,
            image_key="pt",
            image_threshold=0,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[0],
            prob=0.20,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[1],
            prob=0.20,
        ),
        RandFlipd(
            keys=["pt", "seg"],
            spatial_axis=[2],
            prob=0.20,
        ),
        RandRotate90d(
            keys=["pt", "seg"],
            prob=0.20,
            max_k=3,
        ),
    ]
)
val_transforms = Compose(
    [
        LoadImaged(keys=["pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["pt", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["pt",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        #ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
    ]
)

def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)



train_loader = create_dataloader(train_data, train_transforms, shuffle=True)
val_loader = create_dataloader(valid_data, val_transforms, shuffle=False)

In [9]:
import torch
import torch.nn as nn
from monai.networks.nets import UNETR


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the pretrained weights
pretrained_weights = torch.load('/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth')

# Instantiate the new model with 2 input channels
new_model = UNETR(
    in_channels=1, 
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [10]:
torch.backends.cudnn.benchmark = True
def poly_lr(epoch, max_epochs, initial_lr, exponent=0.9):
    return initial_lr * (1 - epoch / max_epochs)**exponent

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(new_model.parameters(), lr=1e-4, weight_decay=1e-5)

In [11]:
model_dir='/home/nada.saadi/CTPET/hecktor2022_cropped/mda-ptonly-PT'
max_iterations = 18000
eval_num = 100

## running stats
global_step = 0
dice_val_best = 0.0
global_step_best = 0
epoch_loss_values = []
metric_values = []
metric_values_tumor = []
metric_values_lymph = []


loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(new_model.parameters(), lr=1e-4, weight_decay=1e-5)


post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch = DiceMetric(include_background=False, reduction="mean_batch")


##-----------------------------------------------------------------------------

def validation(epoch_iterator_val):
        new_model.eval()
        with torch.no_grad():
            for step, batch in enumerate(epoch_iterator_val):
                val_inputs, val_labels = (batch["pt"].cuda(), batch["seg"].cuda())
                val_outputs = sliding_window_inference(val_inputs, (96, 96, 96), 4, new_model)
                val_labels_list = decollate_batch(val_labels)
                val_labels_convert = [
                    post_label(val_label_tensor) for val_label_tensor in val_labels_list
                ]
                val_outputs_list = decollate_batch(val_outputs)
                val_output_convert = [
                    post_pred(val_pred_tensor) for val_pred_tensor in val_outputs_list
                ]
                dice_metric(y_pred=val_output_convert, y=val_labels_convert)
                dice_metric_batch(y_pred=val_output_convert, y=val_labels_convert)
                epoch_iterator_val.set_description(
                    "Validate (%d / %d Steps)" % (global_step, 10.0)
                )
            mean_dice_val = dice_metric.aggregate().item()
            metric_batch_val = dice_metric_batch.aggregate()

            metric_tumor = metric_batch_val[0].item()
            metric_lymph = metric_batch_val[1].item()

            dice_metric.reset()
            dice_metric_batch.reset()
        return mean_dice_val, metric_tumor, metric_lymph



def train(global_step, train_loader, dice_val_best, global_step_best):
        new_model.train()
        epoch_loss = 0
        step = 0
        epoch_iterator = tqdm(
            train_loader, desc="Training (X / X Steps) (loss=X.X)", dynamic_ncols=True
        )
        
        for step, batch in enumerate(epoch_iterator):
            step += 1
            x, y = (batch["pt"].cuda(), batch["seg"].cuda())
            logit_map = new_model(x)
            loss = loss_function(logit_map, y)
            loss.backward()
            epoch_loss += loss.item() 
            optimizer.step()
            optimizer.zero_grad()
            epoch_iterator.set_description(
                "Training (%d / %d Steps) (loss=%2.5f)"
                % (global_step, max_iterations, loss)
            )
            
            if (
                global_step % eval_num == 0 and global_step != 0
            ) or global_step == max_iterations:
                epoch_iterator_val = tqdm(
                    val_loader, desc="Validate (X / X Steps) (dice=X.X)", dynamic_ncols=True
                )
                dice_val, metric_tumor, metric_lymph = validation(epoch_iterator_val)
                epoch_loss /= step
                epoch_loss_values.append(epoch_loss)
                metric_values.append(dice_val)
                metric_values_tumor.append(metric_tumor)
                metric_values_lymph.append(metric_lymph)
                if dice_val > dice_val_best:
                    dice_val_best = dice_val
                    global_step_best = global_step
                    torch.save(
                        new_model.state_dict(), os.path.join(model_dir, "ptonly-PT-with4.pth")
                    )
                    print(
                        "Model Was Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {} Current Avg. tumor Dice: {} Current Avg. lymph Dice: {}".format(
                            dice_val_best, dice_val, metric_tumor, metric_lymph
                        )
                    )
                else:
                    print(
                        "Model Was Not Saved ! Current Best Avg. Dice: {} Current Avg. Dice: {} Current Avg. tumor Dice: {} Current Avg. lymph Dice: {}".format(
                            dice_val_best, dice_val,  metric_tumor, metric_lymph
                        )
                    )
            global_step += 1
        return global_step, dice_val_best, global_step_best

In [12]:
epoch = 0 # used for LR scheduler
max_num_epochs = 530 # used for LR scheduler


while global_step < max_iterations:
    global_step, dice_val_best, global_step_best = train(
        global_step, train_loader, dice_val_best, global_step_best
    )
    epoch += 1
    #optimizer.param_groups[0]['lr'] = poly_lr(epoch, max_num_epochs, 0.005676 , 0.9)
# model.load_state_dict(torch.load(os.path.join(model_dir, "best_metric_luck_UNETr_prompt.pth")))

Training (100 / 18000 Steps) (loss=1.17392):  26%|██▋       | 21/80 [00:38<08:01,  8.15s/it]

Model Was Saved ! Current Best Avg. Dice: 0.15626020729541779 Current Avg. Dice: 0.15626020729541779 Current Avg. tumor Dice: 0.2156621217727661 Current Avg. lymph Dice: 0.10004493594169617


Training (200 / 18000 Steps) (loss=1.10271):  51%|█████▏    | 41/80 [00:46<05:00,  7.70s/it]

Model Was Saved ! Current Best Avg. Dice: 0.19623693823814392 Current Avg. Dice: 0.19623693823814392 Current Avg. tumor Dice: 0.22115392982959747 Current Avg. lymph Dice: 0.17811568081378937


Training (300 / 18000 Steps) (loss=1.03498):  76%|███████▋  | 61/80 [00:55<02:21,  7.42s/it]

...

Training (17800 / 18000 Steps) (loss=0.29326):  51%|█████▏    | 41/80 [00:45<04:46,  7.35s/it]

Model Was Not Saved ! Current Best Avg. Dice: 0.6516297459602356 Current Avg. Dice: 0.6449701189994812 Current Avg. tumor Dice: 0.674564778804779 Current Avg. lymph Dice: 0.6396574378013611


Training (17900 / 18000 Steps) (loss=0.48229):  76%|███████▋  | 61/80 [00:55<02:24,  7.60s/it]

Model Was Saved ! Current Best Avg. Dice: 0.657273530960083 Current Avg. Dice: 0.657273530960083 Current Avg. tumor Dice: 0.6897456645965576 Current Avg. lymph Dice: 0.6490659713745117


Training (17999 / 18000 Steps) (loss=0.48538): 100%|██████████| 80/80 [00:41<00:00,  1.94it/s]
